In [ ]:
# 排列组合子集问题（递归+循环，部分情况可用队列/栈解决）
"""
包括组合问题，排列问题，子集问题，重复元素子集问题（带约束）等
其中求子集是核心，组合是限定长度的子集，排列是限定长度的特殊子集。含重复元素的子集和普通子集代码几近，但必须带有一定约束
"""
# 1 求子集
def subsets(nums):
        res, track = list(), list()  # res用于保存子集，track用于追踪状态
        def backtrack(nums, start): # 递归函数，start为关键参数，“递进”过程中传入i+1 !!!
            res.append(track[:])  # 注意Python的list浅拷贝 切片第一层是深拷贝
            for i in range(start,len(nums)): # 从start出发，避免重复
                track.append(nums[i]) # 选择
                backtrack(nums,i+1) # 递归
                track.pop() # 解除选择
        backtrack(nums,0)
        return res

subsets([1,3,4,7])
"""
以nums = [1,3,4,7]为例，track的状态变化为：
[]
[1]
[1, 3]
[1, 3, 4]
[1, 3, 4, 7]
[1, 3, 7] 关键处：回退+循环
[1, 4]
[1, 4, 7]
[1, 7]
[3]
[3, 4]
[3, 4, 7]
[3, 7]
[4]
[4, 7]
[7]
"""

# 2 重复元素子集（假设约束为子集sum等于4）
def subsets_withdup(nums,target):
        res, track = list(), list()  
        def backtrack(nums, start, tracksum, target): 
            if sum(track)==target: # 结束条件
                res.append(track[:])
                return
            if sum(track)>target:
                return
            
            for i in range(start,len(nums)): 
                track.append(nums[i]); tracksum += nums[i]
                backtrack(nums,i, tracksum, target) # 注意此时start传入i，代表列表元素可重复 ！！
                track.pop(); tracksum -= nums[i]
        backtrack(nums,0,0,target)
        return res

subsets_withdup([1,3,4,7],4)
"""
结果为:[[1, 1, 1, 1], [1, 3], [4]]
"""

# 3 求定长组合
def combine(nums,k):
        res, track = list(), list()  
        def backtrack(nums, start): 
            if len(track)==k: # 限定长度为k
                res.append(track[:]) 
                return

            for i in range(start,len(nums)): 
                track.append(nums[i]) 
                backtrack(nums,i+1) 
                track.pop()
        backtrack(nums,0)
        return res

combine([1,3,4,7],3)
"""
结果为：[[1, 3, 4], [1, 3, 7], [1, 4, 7], [3, 4, 7]]
"""

# 4 求定长排列
def permute(nums,k):
        res = []
        def backtrack(nums, track, used): # used用于表示nums中各元素选择情况
            if len(track)==k: # 限定长度为k
                res.append(track[:]) 
                return
            
            for i in range(len(nums)):
                if used[i]:
                    continue # 强制执行下一个迭代，跳过当前迭代下后面的执行
                track.append(nums[i]); used[i] = True
                backtrack(nums, track, used)
                track.pop(); used[i] = False 
        backtrack(nums,[],[False for _ in range(len(nums))]) # 初始时路径为空，所有元素都没有选择过所以used中都是False
        return res

permute([1,3,4,7],3)
"""
结果为：
[[1, 3, 4], [1, 3, 7], [1, 4, 3], [1, 4, 7], [1, 7, 3], [1, 7, 4],
 [3, 1, 4], [3, 1, 7], [3, 4, 1], [3, 4, 7], [3, 7, 1], [3, 7, 4],
 [4, 1, 3], [4, 1, 7], [4, 3, 1], [4, 3, 7], [4, 7, 1], [4, 7, 3],
 [7, 1, 3], [7, 1, 4], [7, 3, 1], [7, 3, 4], [7, 4, 1], [7, 4, 3]]
"""

In [8]:
# 字符串匹配问题（动态规划）
"""
涉及到字符串匹配问题，如两字符串匹配公共序列，正则表达式匹配，编辑距离等，大部分可用动态规划（DP）的思路来解决.其特点是:
当两字符串上开始相互匹配时，其匹配状态随着两指针动态变化!
当然由于本质上还是暴力匹配，因此时间复杂度为O(mn),m和n分别是两子串长度（KMP匹配算法除外）

如果是判断两字符串能否完全匹配，则动态规划矩阵值可为true，false
"""

# 1 连续公共子串
def common_str1(str1, str2):
        m,n = len(str1),len(str2)
        DP = [[0]*(n+1) for i in range(m+1)] # 状态转移矩阵

        # 状态转移过程
        max_length = 0 # 两字符串最长公共子串长度
        sp = 0 # 两字符串达到最长匹配时,str1所在检索位置
        for i in range (1,m+1):
            for j in range(1,n+1):
                if str1[i-1] == str2[j-1]:
                    DP[i][j] = DP[i-1][j-1]+1
                else:
                    DP[i][j] = 0

                if DP[i][j]>max_length:
                    max_length = DP[i][j]
                    sp = i-1
        print('最长公共子串为:'+str1[sp+1-max_length:sp+1])

common_str1('abcddeba','adcbdeabcd')
"""
最长公共子串为:abcd
"""

# 2 非连续公共子串
def common_str2(str1, str2):
        m,n = len(str1),len(str2)
        DP = [[0]*(n+1) for i in range(m+1)] # 状态转移矩阵

        # 状态转移过程
        max_length = 0 # 两字符串最长公共子串长度
        sp = 0 # 两字符串达到最长匹配时,str1所在检索位置
        for i in range (1,m+1):
            for j in range(1,n+1):
                if str1[i-1] == str2[j-1]:
                    DP[i][j] = DP[i-1][j-1]+1
                else:
                    DP[i][j] = max(DP[i-1][j], DP[i][j-1])

                if DP[i][j]>max_length:
                    max_length = DP[i][j]
                    sp = i-1
        print('最长公共子串为:'+str1[sp+1-max_length:sp+1])

common_str2('abcddeba','adcbdeabcd')
"""
最长公共子串为:cddeb
"""

# 3 编辑距离问题
"""
编辑距离问题可以说是匹配问题的逆向问题，求的是将str1转化为str2所需要的最少编辑次数（编辑包括插入字符，删除字符，替换字符）
"""
def EditDistance(str1, str2):
    m,n = len(str1),len(str2)
    dp = [[0]*(n+1) for _ in range(m+1)]

    # 编辑距离问题中，动态规划矩阵值表示未匹配的次数，最外围则是用str1或str2匹配空字符串，故而矩阵值逐个+1
    for i in range(1,m+1):
        dp[i][0] = dp[i-1][0]+1
    for j in range(1,n+1):
        dp[0][j] = dp[0][j-1]+1

    for i in range(1,m+1):
        for j in range(1,n+1):
            if str1[i-1]==str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j],dp[i][j-1],dp[i-1][j-1])+1
    
    return dp[-1][-1]

EditDistance('abcddeba','adcbdeabcd')
"""
编辑距离为 5
"""

最长公共子串为:abcd


In [2]:
# 双指针问题（包括同向双指针，对向双指针）
# （1）同向双指针问题
"""
同向双指针问题大体上分为滑动窗口问题和快慢指针问题：
前者指left和right指针从0出发，right先行，left后行，两者的区间内为满足给定条件的元素
快慢指针典型问题为判断链表是否构成环，当快指针与慢指针相遇时，说明成环
"""
# 移动零问题
def moveZeroes(nums):
    # 双指针法，指针i用于遍历并定位非零，同时与指针j交换，最后指针j将后面元素全部置零，这也是一种常见的原地修改数组的方法
    m = len(nums)
    i,j = 0,0
    while i<m:
        if nums[i]!=0:
            nums[j] = nums[i]
            j += 1
        i += 1

    while j<m:
        nums[j] = 0
        j += 1

    return nums

# 最小覆盖子串问题
import collections
def minWindow(s, t):
    """
    本题是同向双指针 - 滑动窗口的典型题目，其通用的解题思路如下：
    1 初始化双指针left和right。【初始化】
    2 先不断增加right扩大窗口，直到窗口中的内容符合要求。【寻找可行解】
    3 停止增加right，不断增加left缩小窗口，直到窗口中的内容不再满足要求。
        在每次增加left时都要更新所求的结果。【优化可行解】
    4 不断重复扩大窗口、缩小窗口的操作，直到right到达序列的末端。
    本题在重复2，3步骤时，需要用字典来维护窗口内容
    """ 
    if len(t)>len(s): return ''
    Need = collections.Counter(t) # 记录需匹配的t中各元素数目
    needcnt  = len(t) # 记录需匹配的t中元素个数，若为0表示成功匹配

    start,end = 0,-1 # 符合要求的子串起点和终点
    left,right = 0,0 # 左右滑动指针
    min_len = len(s)+1

    while right<len(s):
        rr = s[right]
        """
        如果新加入窗口中的元素隶属于t，则若原字典Need中ss对应值大于0，表示对ss的匹配还有需求（即还需要考察更多元素），那么自然needcnt-1，并且Need[ss]-1
        """
        if rr in Need: 
            if Need[rr]>0:
                needcnt -= 1
            Need[rr] -= 1
        right += 1
        """
        当needcnt=0时，表明左右指针的滑动窗口包含t中所有元素，但这个滑动窗口是最小的嘛？
        不一定，此时还需要移动左指针，收缩滑动窗口
        """
        while needcnt==0:
            # 更新最小窗口长度
            if right-left<min_len: # 注意此时right向右多滑了一段
                min_len = right-left
                start,end = left,right-1 # 保存最小窗口起点终点
            
            ll = s[left]
            if ll in Need:
                if Need[ll]>=0: # 如果滑出的元素属于t并且造成不匹配
                    needcnt += 1
                Need[ll] += 1
            left += 1

    return s[start:end+1]



# 链表内是否含环问题（此处我们不定义链表）
def hasCycle(head):
    # 快指针走两步，慢指针走一步，快慢指针相遇，则存在环
    slow = fast =head
    while fast and fast.next: # 当不存在环时，fast和fast.next终究会为None
        fast = fast.next.next
        slow = slow.next
        if fast == slow:
            return True
    return False



# （2）对向双指针问题
"""
对向双指针一般用于时间复杂度为log(n)的数组查找问题中，指针从两头出发，向中间收缩，令mid=(left+right)/2，
根据mid所处位置来调整left和right，直至mid达到要求
"""
# 旋转排序数组查找问题
# 解法一：递归，不考虑nums[mid]与target的大小关系，直接暴力dfs
def search1(nums,target):
    """
    设置左右节点left和right，取mid=(left+right)//2 将nums[mid]与target做对比
    """
    l = len(nums)
    res = [-1]
    def dfs(target,left,right):
        if left<=right:
            mid = (left+right)//2
            if nums[mid]==target:
                res.append(mid)
                return 
            dfs(target,mid+1,right)
            dfs(target,left,mid-1)
    dfs(target,0,l-1)
    return res[len(res)-1]

# 解法二：mid与target比较大小，时间复杂度更低
def search2(nums,target):
    if not nums: return -1
    left,right = 0, len(nums)-1
    while left<=right:
        mid = (left+right)//2
        if nums[mid]==target:
            return mid
        if nums[left]<nums[mid]:
            if nums[left]<target<nums[mid]:
                right = mid-1
            else:
                left = mid+1
        else:
            if nums[mid]<target<nums[right]:
                left = mid+1
            else:
                right = mid-1
    return -1

# 旋转排序数组是指非降序数组前面一部分元素被放在后面
nums = [4,5,6,7,8,1,2,3]
target = 6
print(search1(nums,target))
print(search2(nums,target))
"""
输出下标结果为：2
"""

2
2


In [1]:
# 两数组（链表）问题
"""
合并两数组（链表），求两数组中位数，两链表最值等等，也是一类常见的问题，此类问题需要考虑如何合并两数组（链表）
"""
# 寻找两个非降序数组的中位数（合并且保序）
def findMedian(nums1 , nums2):
    m,n = len(nums1),len(nums2)
    length = int((m+n)/2)+1
    list = []
    i,j = 0,0
    while len(list)<length:
        # 如果一个数组为空，则list等于另一个数组
        if m==0:
            list.append(nums2[j])
            j += 1 
        
        if n==0:
            list.append(nums1[i])
            i += 1

        # 两数组同时遍历，直到某一数组遍历完成
        if i<m-1 and j<n-1:
            if nums1[i]<=nums2[j]:
                list.append(nums1[i])
                i += 1
            else:
                list.append(nums2[j])
                j += 1

        # 当某一数组遍历至尾元素时，将其与另一数组剩余元素比较
        if i==m-1 and j<n-1:
            if nums1[i]<=nums2[j]:
                list.append(nums1[i])
                nums1[i] = 1001
            else:
                list.append(nums2[j])
                j += 1

        
        if j==n-1 and i<m-1:
            if nums1[i]<=nums2[j]:
                list.append(nums1[i])
                i += 1
            else:
                list.append(nums2[j])   
                nums2[j] = 1001
        
        # 若同时遍历完成，则比较两者尾元素
        if j==n-1 and i==m-1:
            if nums1[i]<=nums2[j]:
                list.append(nums1[i])
                list.append(nums2[j]) 
            else:
                list.append(nums2[j]) 
                list.append(nums1[i])  

    # 求中位数
    if (m+n)%2==0:
        return (list[length-1]+list[length-2])/2
    else:
        return list[length-1]

findMedian([1,3,4,5],[2,6])
"""
合并后数组为[1,2,3,4,5,6]，中位数为3.5
"""

# 合并两个升序链表
def mergeTwoLists(head1, head2):
    result = current = ListNode() # 新建链表时，为了从头节点返回新链表，通常需要新建两个一样的链表
    while head1 and head2: # 当任一链表遍历完毕时，中断迭代（与上面数组的合并思路类似）
        if head1.val < head2.val:
            current.next,head1 = head1,head1.next
        else:
            current.next,head2 = head2,head2.next
        current = current.next
    current.next = head1 if head1 else head2 # 补全剩下的节点
    return result.next

3.5


In [ ]:
# 二叉树问题
"""

"""